# Setup and data loading

In [ ]:
import sys
sys.path.append("../")  # path contains python_file.py

In [2]:
import iris

import numpy as np
import scipy.stats

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
era5 = iris.load("testing_data/ERA5.nc")
obs = era5[1]

cm_hist = iris.load_cube(
    "testing_data/CMIP6/medium_area/historical_monthly_surface_temp_single_levels_IPSL-CM5A2-INCA/tas_Amon_IPSL-CM5A2-INCA_historical_r1i1p1f1_gr_19790116-20141216_v20200729.nc"
)
cm_future = iris.load_cube(
    "testing_data/CMIP6/medium_area/SSP3-7_monthly_surface_temp_single_levels_IPSL-CM5A2-INCA/tas_Amon_IPSL-CM5A2-INCA_ssp370_r1i1p1f1_gr_20150116-20501216_v20201218.nc"
)

/home/jakobwes/anaconda3/lib/python3.8/site-packages/iris/fileformats/cf.py:859: UserWarning: Missing CF-netCDF measure variable 'areacella', referenced by netCDF variable 'tas'
  warnings.warn(
/home/jakobwes/anaconda3/lib/python3.8/site-packages/iris/fileformats/cf.py:859: UserWarning: Missing CF-netCDF measure variable 'areacella', referenced by netCDF variable 'tas'
  warnings.warn(


In [5]:
# Regrid
cm_hist = cm_hist.regrid(obs, iris.analysis.Linear())
cm_future = cm_future.regrid(obs, iris.analysis.Linear())

In [6]:
# Get data
obs = obs.data[:, 0, :, :]
cm_hist = cm_hist.data
cm_future = cm_future.data

# Show debiasers running through

In [8]:
from scipy.stats import norm, laplace

from PACKAGE_NAME.debias import QuantileMapping, QuantileDeltaMapping, ScaledDistributionMapping, LinearScaling, EquidistantCDFMatching

distribution = norm

temp_debiaser = QuantileMapping(distribution=norm, delta_type="no_delta")
debiased_cm1 = temp_debiaser.apply(obs, cm_hist, cm_future)

temp_debiaser = QuantileMapping.from_variable(variable="temp")
debiased_cm2 = temp_debiaser.apply(obs, cm_hist, cm_future)

temp_debiaser = QuantileMapping.from_variable(variable="temp", delta_type="additive", distribution = laplace)
debiased_cm3 = temp_debiaser.apply(obs, cm_hist, cm_future)

temp_debiaser = EquidistantCDFMatching.from_variable(variable = "temp", distribution = norm)
debiased_cm4 = temp_debiaser.apply(obs, cm_hist, cm_future)

temp_debiaser = QuantileDeltaMapping(distribution=norm, time_window_length=50)
debiased_cm5 = temp_debiaser.apply(obs, cm_hist, cm_future)

temp_debiaser = ScaledDistributionMapping.from_variable(variable = "temp")
debiased_cm6 = temp_debiaser.apply(obs, cm_hist, cm_future)

temp_debiaser = LinearScaling(delta_type = "additive")
debiased_cm7 = temp_debiaser.apply(obs, cm_hist, cm_future)

----- Running debiasing -----


100%|██████████| 357/357 [00:00<00:00, 2161.51it/s]


----- Running debiasing -----


100%|██████████| 357/357 [00:00<00:00, 1797.68it/s]


----- Running debiasing -----


...

100%|██████████| 357/357 [00:09<00:00, 39.25it/s]


----- Running debiasing -----


100%|██████████| 357/357 [00:00<00:00, 691.20it/s]


----- Running debiasing -----


100%|██████████| 357/357 [00:00<00:00, 6176.16it/s]


# Show different ways to initialize debiasers

In [9]:
from scipy.stats import norm, laplace, gamma

from PACKAGE_NAME.debias import QuantileMapping
from PACKAGE_NAME.debias import QuantileDeltaMapping
from PACKAGE_NAME.variables import Temperature, Precipitation
from PACKAGE_NAME.utils import gen_PrecipitationGammaLeftCensoredModel, gen_PrecipitationHurdleModel

# Using a string and the required argument: maps to default settings
temp_debiaser1 = QuantileMapping.from_variable(variable = "temp", delta_type="no_delta")

# Using a string and the required argument: maps to default settings
temp_debiaser2 = QuantileMapping.from_variable(variable = "tas", delta_type="additive")

# Using the variable class: maps to default settings
temp_debiaser3 = QuantileMapping.from_variable(variable=Temperature, delta_type="additive")

# Using the variable class but overwriting the standard distribution
temp_debiaser4 = QuantileMapping.from_variable(variable=Temperature, delta_type="additive", distribution = laplace)

# Using special initialisation for precipitation
precip_debiaser1 = QuantileMapping.for_precipitation(delta_type = "multiplicative", precipitation_model_type = "censored", precipitation_censoring_value = 0.1)

# Specifying all the arguments and an own model for the distribution (child-class of StatisticalModel needing only fit, cdf and ppf method) 1
left_censored_model = gen_PrecipitationGammaLeftCensoredModel(censoring_value = 0.1)
precip_debiaser2 = QuantileMapping(delta_type = "multiplicative", distribution = left_censored_model)

# Using the for_precipitation method but an own model for the distribution (child-class of StatisticalModel needing only fit, cdf and ppf method) 2
hurdle_model = gen_PrecipitationHurdleModel(distribution = gamma)
precip_debiaser3 = QuantileMapping.for_precipitation(delta_type = "multiplicative", distribution = hurdle_model)

# Using a variable, but overwriting the distribution
hurdle_model = gen_PrecipitationHurdleModel(distribution = gamma)
precip_debiaser4 = QuantileMapping.from_variable(variable = "pr", delta_type = "multiplicative", distribution = hurdle_model)


In [10]:
temp_debiaser1 == temp_debiaser2 # no because of different delta_type

False

In [11]:
temp_debiaser2 == temp_debiaser3 # yes

True

In [12]:
temp_debiaser3 == temp_debiaser4 # no because of different distribution

False

In [13]:
precip_debiaser1 == precip_debiaser2 # yes because same models, just differently specified

True

In [14]:
precip_debiaser2 == precip_debiaser3 # no because of different model: hurdle vs. censored

False

In [15]:
precip_debiaser3 == precip_debiaser4 # yes because same model, just differently specified

True